In [ ]:
import sqlite3
import pandas as pd
import time

st = time.time()

conn = sqlite3.connect("stock.db")
ds_ratio = pd.read_sql_query('select STOCK_CODE, STOCK_FACTOR, STOCK_YEAR, STOCK_DATA from VCSC_FINANCIAL_RATIO', conn)
ds_price = pd.read_sql_query('select STOCK_CODE, STOCK_DATE, STOCK_CLOSE from VCSC_STOCK_PRICE', conn)
conn.close()

print("1--- %s seconds ---" % (time.time() - st))
st = time.time()

#ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'EPS (VND)']
ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'BVS']

ds_ratio = ds_ratio.loc[ds_ratio['STOCK_YEAR'].str.contains('TTM')==False]
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace('-', '')
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace(',', '')
ds_ratio['STOCK_DATA'] = pd.to_numeric(ds_ratio['STOCK_DATA'])
ds_ratio['STOCK_YEAR'] = pd.to_numeric(ds_ratio['STOCK_YEAR'])

print("2--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price['STOCK_DATE'] = pd.to_datetime(ds_price['STOCK_DATE'], format='%m/%d/%Y')
ds_price['STOCK_YEAR'] = ds_price['STOCK_DATE'].dt.year
ds_price['STOCK_QUARTER'] = ds_price['STOCK_YEAR'].astype('str') + ds_price['STOCK_DATE'].dt.quarter.astype('str')
ds_price['STOCK_CLOSE'] = ds_price['STOCK_CLOSE'].str.replace(',', '').astype(float)

print("3--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_QUARTER']).nth(0)
ds_price['NEXT-PRICE-Q'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)
ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_YEAR']).nth(0)
ds_price['NEXT-PRICE-Y'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)

print("4--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_ratio = ds_ratio.merge(ds_price, left_on=['STOCK_CODE', 'STOCK_YEAR'], right_on=['STOCK_CODE', 'STOCK_YEAR'], how='right')
ds_ratio['EP'] = ds_ratio['STOCK_DATA'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['RETURN'] = ds_ratio['NEXT-PRICE-Y'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['DECILE'] = (ds_ratio.groupby('STOCK_YEAR')['EP']
                                .apply(lambda x: pd.qcut(x,10,duplicates='drop',labels=False)))
s1 = ds_ratio.groupby(['STOCK_YEAR','DECILE'], as_index=False).mean()

print("5--- %s seconds ---" % (time.time() - st))
st = time.time()

s1.pivot(index='DECILE', columns='STOCK_YEAR', values='RETURN')
